# 말로하는 지도 만들기라는 거창한 프로젝트를 위해... DB와의 통신이 필요해졌기에 
# Quickstart
- 이 가이드에서는 SQL 데이터베이스를 사용하여 Q&A 체인과 에이전트를 만드는 기본적인 방법을 살펴볼 것입니다. 
- 이 시스템들을 통해 SQL 데이터베이스에 대한 질문을 하고 자연어 답변을 받을 수 있습니다. 
- 두 시스템의 주요 차이점은 에이전트가 질문에 답하기 위해 필요한 만큼 데이터베이스를 반복적으로 쿼리할 수 있다는 것입니다.
# ⚠️ Security note ⚠️
- SQL 데이터베이스의 Q&A 시스템을 구축하는 것은 모델이 생성한 SQL 쿼리를 실행해야 합니다. 
- 이 과정에는 본질적인 위험이 따릅니다. 
- 데이터베이스 연결 권한을 항상 체인/에이전트의 필요에 맞게 가능한 한 좁게 설정해야 합니다. 
- 이는 위험을 완전히 제거하지는 않지만 줄일 수 있습니다. 일반적인 보안 모범 사례에 대해서는 여기를 참조하세요.
# Architecture
- 전반적으로, 모든 SQL 체인과 에이전트의 단계는 다음과 같습니다:
- 질문을 SQL 쿼리로 변환: 모델이 사용자 입력을 SQL 쿼리로 변환합니다.
- SQL 쿼리 실행: SQL 쿼리를 실행합니다.
- 질문에 답하기: 모델이 쿼리 결과를 사용하여 사용자 입력에 응답합니다.


In [10]:
from dotenv import load_dotenv
load_dotenv('../dot.env')
import os
import getpass


def _set_if_undefined(var: str):
    # 주어진 환경 변수가 설정되어 있지 않다면 사용자에게 입력을 요청하여 설정합니다.
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"Please provide your {var}")


# OPENAI_API_KEY 환경 변수가 설정되어 있지 않으면 사용자에게 입력을 요청합니다.
_set_if_undefined("OPENAI_API_KEY")
# LANGCHAIN_API_KEY 환경 변수가 설정되어 있지 않으면 사용자에게 입력을 요청합니다.
_set_if_undefined("LANGCHAIN_API_KEY")
# TAVILY_API_KEY 환경 변수가 설정되어 있지 않으면 사용자에게 입력을 요청합니다.
_set_if_undefined("TAVILY_API_KEY")

# LangSmith 추적 기능을 활성화합니다. (선택적)
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "QA_SQL_CSV_Quickstart"



# 로컬에 설치한 DB와의 연결

In [11]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artist LIMIT 10;")

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

# 

# sql문 리턴 성능
- gpt-3.5-turbo: 완벽
- xionic-ko-llama-3-70b: 미달
- QuantFactory/dolphin-2.9-llama3-8b-GGUF: 상(프롬프트 조작을 통해 사용 가능)
- mradermacher_f16/Joah-Remix-Llama-3-KoEn-8B-Reborn-GGUF: 상(프롬프트 조작을 통해 사용 가능)
- asiansoul_q8_0/Joah-Remix-Llama-3-KoEn-8B-Reborn-8B-Q8_0: 상(프롬프트 조작을 통해 사용 가능)

In [13]:
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.prompts import PromptTemplate
from langchain import hub
from langchain_core.output_parsers import StrOutputParser


# llm = ChatOpenAI(model="gpt-3.5-turbo",
#                 temperature=0,
#                 streaming=True,
#     callbacks=[StreamingStdOutCallbackHandler()]
#                 )


# llm = ChatOpenAI(
#     base_url="http://localhost:1234/v1", #-> lmstudio를 통해 열어놓은 서버로 llm을 구동하고 있는 상태, lmstudio에 서빙하고 있는 모델만 바꿔주면 새로 나온 모델을 시험해볼 수 있음
#     api_key="lm-studio",
#     model="mradermacher_f16/Joah-Remix-Llama-3-KoEn-8B-Reborn-GGUF",
#     temperature=0,
#     streaming=True,
#     callbacks=[StreamingStdOutCallbackHandler()]
# )

llm = ChatOpenAI(
    base_url="http://localhost:1234/v1", #-> lmstudio를 통해 열어놓은 서버로 llm을 구동하고 있는 상태, lmstudio에 서빙하고 있는 모델만 바꿔주면 새로 나온 모델을 시험해볼 수 있음
    api_key="lm-studio",
    model="asiansoul_q8_0/Joah-Remix-Llama-3-KoEn-8B-Reborn-8B-Q8_0",
    temperature=0,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

# llm = ChatOpenAI(
#     base_url = 'http://sionic.chat:8001/v1',
#     api_key = "934c4bbc-c384-4bea-af82-1450d7f8128d",
#     model = 'xionic-ko-llama-3-70b',
#     temperature = 0.1,
#     streaming=True,
#     callbacks=[StreamingStdOutCallbackHandler()]
# )

# llm = ChatOpenAI(
#     base_url="http://localhost:1234/v1", #-> lmstudio를 통해 열어놓은 서버로 llm을 구동하고 있는 상태, lmstudio에 서빙하고 있는 모델만 바꿔주면 새로 나온 모델을 시험해볼 수 있음
#     api_key="lm-studio",
#     model="QuantFactory/dolphin-2.9-llama3-8b-GGUF",
#     temperature=0,
#     streaming=True,
#     callbacks=[StreamingStdOutCallbackHandler()]
# )
# ----------------------------------------------
prompt = hub.pull("rlm/text-to-sql")

# Create chain with LangChain Expression Language
inputs = {
    "table_info": lambda x: db.get_table_info(),
    "input": lambda x: x["question"],
    "few_shot_examples": lambda x: "",
    "dialect": lambda x: db.dialect,
}
# ----------------------------------------------

sql_response = (
    inputs
    | prompt
    | llm.bind(stop=["SQLResult:"])
    | StrOutputParser()
    | (lambda x: x.split("SQLQuery: ")[1]) #LCED의 lambda 사용법 익히기
)

# Call with a given question
response = sql_response.invoke({"question": "How many customers are there?"})
# chain = create_sql_query_chain(llm, db) #-> db정보를 context로 주는 chain, connection을 이용, 실시간 db에 접근할 수 있는지
# query = 'What information is in the employee table?'
# # response = chain.invoke({"question": f" just return sql query about {query}"})
# response = chain.invoke({"question": f"{query}"})
print(response)
db.run(response)


SQLQuery: SELECT COUNT(*) FROM Customer;
SELECT COUNT(*) FROM Customer;



'[(59,)]'

# 로컬모델을 이용하더라도 프롬프토 조작을 통해 원하는 답변 얻을 수 있음

# LangSmith 추적을 통해 이 체인이 무엇을 하는지 더 잘 이해할 수 있습니다. 또한 체인의 프롬프트를 직접 검사할 수도 있습니다. 아래의 프롬프트를 보면 다음과 같습니다:

- 방언 특정입니다. 이 경우에는 SQLite를 명시적으로 참조합니다.
- 사용 가능한 모든 테이블에 대한 정의가 있습니다.
- 각 테이블에 대해 세 개의 예제 행이 있습니다.
- 이 기술은 예제 행을 보여주고 테이블에 대해 명시적으로 설명하는 것이 성능을 향상시킨다고 제안하는 논문에서 영감을 받았습니다. 전체 프롬프트를 다음과 같이 검사할 수도 있습니다:


LangChain docs 와는 달리 아래 prompt와 inputs를 사용, llama-3 기반 오픈
모델으로도 쿼리 생성 및 실행 가능
---------------------------
- prompt(hub.pull("rlm/text-to-sql"))
- inputs = {
    "table_info": lambda x: db.get_table_info(),
    "input": lambda x: x["question"],
    "few_shot_examples": lambda x: "",
    "dialect": lambda x: db.dialect,
}
-----------------------------


In [6]:
# 프롬프트 검사
sql_response.get_prompts()[0].pretty_print()

================================ Human Message =================================

Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

Only use the following tables:

{table_info}.

Some examples of SQL queries that corrsespond to questions are:

{few_shot_examples}

Question: {input}


# 쿼리 실행기 추가
- QuerySQLDataBaseTool을 Chain에 넣어 생성된 sql query를 곧바로 실행할 수 있음.

In [14]:
from langchain_community.tools import QuerySQLDataBaseTool

execute_query = QuerySQLDataBaseTool(db = db)

query = '''how many employee are there?'''
response = sql_response

chain = response | execute_query
chain.invoke({'question': f'{query}'})



SQLQuery: SELECT COUNT(*) FROM Employee;


'[(8,)]'

# 질문에 답하기
- 이제 자동으로 SQL 쿼리를 생성하고 실행하는 방법을 갖추었으므로, 원래의 질문과 SQL 쿼리 결과를 결합하여 최종 답변을 생성하기만 하면 됩니다. 이를 위해 질문과 결과를 다시 LLM에 전달할 수 있습니다:


In [15]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

answer = answer_prompt | llm | StrOutputParser()
chain = (
    RunnablePassthrough.assign(query=response).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)

chain.invoke({"question": "How many employees are there"})

SQLQuery: SELECT COUNT(*) FROM Employee;
8명의 직원이 있습니다.

'8명의 직원이 있습니다.'

# Chain을 구성하는 방법 외에 agent를 구성하는 방법도 있습니다

LangChain에는 SQL 데이터베이스와 상호 작용하는 더 유연한 방법을 제공하는 SQL 에이전트가 있습니다. SQL 에이전트를 사용하는 주요 장점은 다음과 같습니다:
 
- 데이터베이스의 스키마뿐만 아니라 데이터베이스의 내용(특정 테이블 설명 등)을 기반으로 질문에 답할 수 있습니다.
- 생성된 쿼리를 실행하고 트레이스백을 잡아 올바르게 재생성하여 오류에서 복구할 수 있습니다.
- 여러 종속 쿼리가 필요한 질문에 답할 수 있습니다.
- 관련 테이블의 스키마만 고려하여 토큰을 절약할 수 있습니다.
- 에이전트를 초기화하려면 create_sql_agent 함수를 사용합니다. 이 에이전트에는 다음과 같은 도구를 포함하는 SQLDatabaseToolkit이 포함되어 있습니다:
 
- 쿼리 생성 및 실행
- 쿼리 구문 검사
- 테이블 설명 검색


In [24]:
from langchain_community.agent_toolkits import create_sql_agent

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)
agent_executor = create_sql_agent(llm=llm, db=db, agent_type="openai-tools", verbose=True)


In [26]:
agent_executor.invoke(
    {
        "input": "List the total sales per all countries. Which country's customers spent the most?"
    }
)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Invoking: `sql_db_schema` with `{'table_names': 'Customer, Invoice, InvoiceLine'}`



CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60) NOT NULL, 
	"SupportRepId" INTEGER, 
	PRIMARY KEY ("CustomerId"), 
	FOREIGN KEY("SupportRepId") REFERENCES "Employee" ("EmployeeId")
)

/*
3 rows from Customer table:
CustomerId	FirstName	LastName	Company	Address	City	State	Country	PostalCode	Phone	Fax	Email	SupportRepId
1	Luís	Gonçalves	Embraer - Empresa Brasileira de Aeronáutica S.A.	Av. Brigadeiro Faria Lima, 

{'input': "List the total sales per all countries. Which country's customers spent the most?",
 'output': 'The total sales per country are as follows:\n\n1. USA: $523.06\n2. Canada: $303.96\n3. France: $195.10\n4. Brazil: $190.10\n5. Germany: $156.48\n6. United Kingdom: $112.86\n7. Czech Republic: $90.24\n8. Portugal: $77.24\n9. India: $75.26\n10. Chile: $46.62\n\nThe country whose customers spent the most is the USA with a total sales amount of $523.06.'}

In [27]:
agent_executor.invoke({"input": "Describe the playlisttrack table"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Invoking: `sql_db_schema` with `{'table_names': 'PlaylistTrack'}`



CREATE TABLE "PlaylistTrack" (
	"PlaylistId" INTEGER NOT NULL, 
	"TrackId" INTEGER NOT NULL, 
	PRIMARY KEY ("PlaylistId", "TrackId"), 
	FOREIGN KEY("TrackId") REFERENCES "Track" ("TrackId"), 
	FOREIGN KEY("PlaylistId") REFERENCES "Playlist" ("PlaylistId")
)

/*
3 rows from PlaylistTrack table:
PlaylistId	TrackId
1	3402
1	3389
1	3390
*/The `PlaylistTrack` table has the following columns:
- PlaylistId: INTEGER (NOT NULL)
- TrackId: INTEGER (NOT NULL)

It is a junction table with a composite primary key consisting of PlaylistId and TrackId. The table also has foreign key constraints referencing the `Track` table and the `Playlist` table.

Here are 3 sample rows from the `PlaylistTrack` table:
1. PlaylistId: 1, TrackId: 3402
2. Pl

{'input': 'Describe the playlisttrack table',
 'output': 'The `PlaylistTrack` table has the following columns:\n- PlaylistId: INTEGER (NOT NULL)\n- TrackId: INTEGER (NOT NULL)\n\nIt is a junction table with a composite primary key consisting of PlaylistId and TrackId. The table also has foreign key constraints referencing the `Track` table and the `Playlist` table.\n\nHere are 3 sample rows from the `PlaylistTrack` table:\n1. PlaylistId: 1, TrackId: 3402\n2. PlaylistId: 1, TrackId: 3389\n3. PlaylistId: 1, TrackId: 3390'}